In [1]:
import time
import functools
import asyncio
from asyncio.coroutines import iscoroutine

### Sync function

In [4]:
def typical_sync_function(n):
    time.sleep(n)
    return f"I sleep for {n} seconds."

In [5]:
print(typical_sync_function(10))

I sleep for 10 seconds.


### More complicated

In [7]:
def call_youtube():
    time.sleep(5)

def call_database():
    time.sleep(5)
    
def call_external_url():
    time.sleep(5)
    
def typical_sync_function(some_data):
    print("calling youtube")
    call_youtube()
    print("calling database")
    call_database()
    print("calling external_url")
    call_external_url()
    return f"I sleep for {15} seconds."

In [8]:
print(typical_sync_function(12))

calling youtube
calling database
calling external_url
I sleep for 15 seconds.


In [10]:
def concat_sequence_v1(s1, s2):
    for elem in s1:
        yield elem
    for elem in s2:
        yield elem


def concat_sequence_v2(s1, s2):
    yield from s1
    yield from s2        

In [11]:
seq1 = range(10)
seq2 = range(10, 20)
result = concat_sequence_v1(seq1, seq2)

# print(f'Seq 1 {list(result)}')
for i in result:
    print(i)

# result = concat_sequence_v2(seq1, seq2)

# print(f'Seq 2 {list(result)}')
# for i in result:
#     print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [14]:
def is_divider(number):
    print("Coroutine started")
    while True:
        value = yield
        if number % value == 0:
            print(value)

cor = is_divider(100)
# !!!
cor.send(None)

cor.send(11)
cor.send(10)
cor.send(18)
cor.send(20)
cor.close()

Coroutine started
10
20


In [28]:
def coroutine(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        res.send(None)
        return res

    return wrapper


@coroutine
def is_divider_cor(number):
    print("Coroutine started")
    while True:
        value = yield
        if number % value == 0:
            print(value)


cor = is_divider_cor(100)
# No need to send None
cor.send(10)
cor.send(10)
cor.send(10)
cor.close()


Coroutine started
10
10
10


In [30]:
print(iscoroutine(is_divider_cor))

False


In [18]:
def sync_worker(number, divider):
    print('Sync Worker started with values: {} / {}'.format(number, divider))
    time.sleep(1)
    print(number / divider)


@asyncio.coroutine
def async_worker(number, divider):
    print('Async Worker started with values: {} / {}'.format(number, divider))
    yield from asyncio.sleep(3)
    print(number / divider)


# sync
sync_worker(30, 10)
sync_worker(20, 10)

print(iscoroutine(sync_worker))
print(iscoroutine(async_worker(10, 2)))


event_loop = asyncio.get_event_loop()
task_list = [
    event_loop.create_task(async_worker(30, 10)),
    event_loop.create_task(async_worker(50, 25)),
]
tasks = asyncio.wait(task_list)
# event_loop.run_until_complete(tasks)
# event_loop.close()


C:\Users\dell\AppData\Local\Temp/ipykernel_39332/4080191993.py:8: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def async_worker(number, divider):


Sync Worker started with values: 30 / 10
3.0
Sync Worker started with values: 20 / 10
2.0
False
True
Async Worker started with values: 30 / 10
Async Worker started with values: 50 / 25
3.0
2.0


In [19]:
async def async_worker(number, divider):
    print('Worker {} started'.format(number))
    await asyncio.sleep(2)
    print(number / divider)
    return number / divider


async def gather_worker():
    result = await asyncio.gather(
        async_worker(50, 10),
        async_worker(60, 10),
        async_worker(70, 10),
        async_worker(80, 10),
        async_worker(90, 10),
    )
    print(result)


event_loop = asyncio.get_event_loop()
task_list = [
    event_loop.create_task(gather_worker())
]
tasks = asyncio.wait(task_list)
# event_loop.run_until_complete(tasks)
# event_loop.close()

# 5.0
# 7.0
# 9.0
# 6.0
# 8.0

C:\Users\dell\AppData\Local\Temp/ipykernel_39332/3011935161.py:23: RuntimeWarning: coroutine 'wait' was never awaited
  tasks = asyncio.wait(task_list)


Worker 50 started
Worker 60 started
Worker 70 started
Worker 80 started
Worker 90 started
5.0
7.0
9.0
6.0
8.0
[5.0, 6.0, 7.0, 8.0, 9.0]


In [50]:
async def async_worker(seconds):
    print('Sleep using {}'.format(seconds))
    await asyncio.sleep(seconds)
    print('Done sleep: {}'.format(seconds))


async def stop_event_loop(loop, seconds):
    print('Stop in {}s'.format(seconds))
    await asyncio.sleep(seconds)
    loop.stop()
    print('Stopped')


async def resolve_future(future):
    await asyncio.sleep(5)
    print('Future set_result')
    future.set_result(10)


async def wait_for_future(future):
    result = await future
    print('Future result: {}'.format(result))


event_loop = asyncio.get_event_loop()

# создаем футур и передадим его в корутину.
# в другой корутине будем дожидаться рещультата от данного футура.
# fut = event_loop.create_future()
fut = asyncio.Future()

# добавляем в цико событий две задачи
event_loop.create_task(async_worker(3))
event_loop.create_task(async_worker(4))

# останавливаем цикл событий, независимо от успещности завершения в нём задач.
event_loop.create_task(stop_event_loop(event_loop, 13))

# в этой задаче мы установим результат футура
event_loop.create_task(resolve_future(fut))

# в этой задаче мы будем ждать результата футура
event_loop.create_task(wait_for_future(fut))

# запускаем бесконечный цикл событий, но так как выше мы добавили задачу
# для его остановки, то через 13 секунд мы выйдем из него.
event_loop.run_forever()
event_loop.close()


C:\Python310\lib\codeop.py:150: RuntimeWarning: coroutine 'wait' was never awaited
  codeob = compile(source, filename, symbol, self.flags, True)


RuntimeError: This event loop is already running

Sleep using 3
Sleep using 4
Stop in 13s
Done sleep: 3
Done sleep: 4
Future set_result
Future result: 10
